# (Gridsearch) 00 LSTM trefor park

In [1]:
import torch
from core.util.hyperparameter_configuration import get_hyperparameter_configuration
from core.util.get_datasets import cross_validation
from core.models.model_training import train_model
from sklearn.model_selection import ParameterGrid
from core.util.save_model import save_parameters
from core.util.trefor_dataset import TreforData
from torch.utils.data import DataLoader
from core.models import LSTM

## Configuration
Parameters specific to this experiment

In [2]:
experiment_name = "00_LSTM_trefor_park"
features = {}
model_input_size = len(features) + 1
model_used = LSTM

All the parameters we do grid search on

In [3]:
gridsearch_params = {
    "learning_rate": [0.001, 0.003, 0.005],
    "batch_size": [32, 64, 128],
    "lookback": [36, 48, 96],
    "num_layers": [1, 2],
    "seed": [1337, 0xC0FFEE, 0xDEADBEEF],
}

Global hyperparameter configuration

In [4]:
hyperparameters = get_hyperparameter_configuration()
hidden_size = hyperparameters["hidden_size"]
epochs = hyperparameters["epochs"]
horizon = hyperparameters["horizon"]
loss_function = hyperparameters["loss_function"]
dropout_rate = hyperparameters["dropout_rate"]
train_days = hyperparameters["train_days"]
val_days = hyperparameters["val_days"]
test_days = hyperparameters["test_days"]
early_stopper = hyperparameters["early_stopper"]

hyperparameters

{'hidden_size': 16,
 'epochs': 200,
 'horizon': 24,
 'loss_function': <function core.util.loss_functions.zap_loss(y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor>,
 'dropout_rate': 0.2,
 'train_days': 16,
 'val_days': 2,
 'test_days': 2,
 'early_stopper': <core.util.early_stop.EarlyStop at 0x7f949f2095b0>}

If the host has CUDA, it will use the GPU for computation

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

## Training
Train a model with specified hyperparameters

In [6]:
def train_with_params(params: dict) -> tuple[float, model_used]:
    """Train model with the specified hyperparameters."""
    # Extract hyperparameters
    batch_size = params["batch_size"]
    learning_rate = params["learning_rate"]
    num_layers = params["num_layers"]
    lookback = params["lookback"]
    torch.manual_seed(params["seed"])

    x_train, y_train, x_val, y_val, x_test, y_test = cross_validation(
        lookback=lookback,
        horizon=horizon,
        train_days=train_days,
        val_days=val_days,
        test_days=test_days,
        features={},
    )

    # convert to dataset that can use dataloaders
    train_dataset = TreforData(x_train, y_train, device)
    val_dataset = TreforData(x_val, y_val, device)

    # initialize the dataloaders, without shuffeling the data between epochs
    training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = model_used(
        input_size=model_input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout_rate=dropout_rate,
        horizon=horizon,
        lookback=lookback,
    ).to(device)

    _, val_loss, best_model = train_model(
        epochs=epochs,
        model=model,
        loss_function=loss_function,
        training_loader=training_loader,
        validation_loader=validation_loader,
        learning_rate=learning_rate,
        early_stopper=early_stopper,
    )

    return min(val_loss), best_model

Iterate over all hyperparameters and train a model for each combination.

In [7]:
results = []
for params in ParameterGrid(gridsearch_params):
    early_stopper.reset()
    v_loss, model = train_with_params(params)
    print(params, v_loss)
    results.append([v_loss, params])

Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 32, 'learning_rate': 0.001, 'lookback': 36, 'num_layers': 1, 'seed': 1337} 0.00813481726223212


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 32, 'learning_rate': 0.001, 'lookback': 36, 'num_layers': 1, 'seed': 12648430} 0.00952484041423304


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

...

{'batch_size': 128, 'learning_rate': 0.005, 'lookback': 96, 'num_layers': 2, 'seed': 1337} 0.005867087693287082


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 128, 'learning_rate': 0.005, 'lookback': 96, 'num_layers': 2, 'seed': 12648430} 0.005575289012703263


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 128, 'learning_rate': 0.005, 'lookback': 96, 'num_layers': 2, 'seed': 3735928559} 0.005839480637618697


Sort the results and save the best 3 parameters

In [8]:
results = sorted(results)
for i in range(3):
    print(f"Best Hyperparameters {i}:", results[i][1])
    print(f"Best Validation Loss {i}:", results[i][0])
    save_parameters(f"{experiment_name}_{i}", results[i][1], results[i][0])

Best Hyperparameters 0: {'batch_size': 128, 'learning_rate': 0.003, 'lookback': 96, 'num_layers': 2, 'seed': 1337}
Best Validation Loss 0: 0.005496167733023564
Best Hyperparameters 1: {'batch_size': 64, 'learning_rate': 0.003, 'lookback': 96, 'num_layers': 2, 'seed': 12648430}
Best Validation Loss 1: 0.005572336480044615
Best Hyperparameters 2: {'batch_size': 128, 'learning_rate': 0.005, 'lookback': 96, 'num_layers': 2, 'seed': 12648430}
Best Validation Loss 2: 0.005575289012703263
